<a href="https://colab.research.google.com/github/lugastyan98/portofolio/blob/main/rekomendation_system_versi_menggunakan_descripsi_dan_string_lainnya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# library

In [ ]:
!pip install rake_nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

import rake_nltk
from rake_nltk import Rake
import nltk
nltk.download('stopwords')
nltk.download('punkt')

from bs4 import BeautifulSoup #untuk perHTMLan

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Load dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/PINA dataset/semua_versi_4___edwin_2023-05-10T04_25_24.475027Z.csv')

In [ ]:
df_user = pd.read_csv('/content/drive/MyDrive/PINA dataset/user_portfolio.csv')

In [ ]:
df_user.head()

,clientportfolioclientid,clientportfoliostockid,clientportfoliobalance,clientportfoliodate,date_rank_per_user
0,0000003,FIMP,19000000.0,2023-05-05,1
1,01OLK,BOLT,100.0,2023-05-05,1
2,3,CASH,47051800.0,2022-03-28,1
3,ADANGWAH107,TLKM,100.0,2023-05-05,1
4,ADIN99,TRIM,100.0,2023-05-05,1


In [ ]:
df_grouped = pd.DataFrame(df_user.groupby('clientportfolioclientid')['clientportfoliostockid'].agg(list))

In [ ]:
df_grouped = df_grouped.reset_index()

In [ ]:
df_grouped.head() # rencanaya ini mau dipake buat user base

,clientportfolioclientid,clientportfoliostockid
0,0000003,[FIMP]
1,01OLK,[BOLT]
2,3,[CASH]
3,ADANGWAH107,[TLKM]
4,ADIN99,[TRIM]


In [ ]:
df_grouped.columns  = ['id','portofolio']

In [ ]:
df_grouped.head()

,id,portofolio
0,0000003,[FIMP]
1,01OLK,[BOLT]
2,3,[CASH]
3,ADANGWAH107,[TLKM]
4,ADIN99,[TRIM]


In [ ]:
df_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555 entries, 0 to 554
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          555 non-null    object
 1   portofolio  555 non-null    object
dtypes: object(2)
memory usage: 8.8+ KB


In [ ]:
print(df_grouped['id'].nunique())

555


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 883 entries, 0 to 882
Data columns (total 54 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   _id                      883 non-null    object 
 1   sector                   883 non-null    object 
 2   idx                      883 non-null    object 
 3   thematic                 883 non-null    object 
 4   general_info             865 non-null    object 
 5   business_product         669 non-null    object 
 6   total_asset_growth       853 non-null    float64
 7   price_sales_ratio        845 non-null    float64
 8   sales_growth             834 non-null    float64
 9   quick_ratio              849 non-null    float64
 10  cost_of_goods_sold       853 non-null    float64
 11  cash_flow_per_share      850 non-null    float64
 12  net_income_growth        839 non-null    float64
 13  operating_profit         853 non-null    float64
 14  income_before_tax        8

# Missing Value

Missing value akan diisi dengan rata-rata.
Tidak dihapus karena data _id itu merupakan data semua saham di PINA


In [ ]:
df.isnull().sum() #cek yang NaNT

_id                          0
sector                       0
idx                          0
thematic                     0
general_info                18
business_product           214
total_asset_growth          30
price_sales_ratio           38
sales_growth                49
quick_ratio                 34
cost_of_goods_sold          30
cash_flow_per_share         33
net_income_growth           44
operating_profit            30
income_before_tax           30
total_assets                30
total_equity                30
total_liabilities           30
debt_to_equity_ratio        30
cash_equivalent             30
total_sales                 30
operating_profit_margin     35
gross_profit_margin         35
net_profit_margin           35
net_income                  30
market_cap                  22
dailyReturn                 22
monthlyReturn               22
annualReturn                22
threeYearReturn             22
fiveYearReturn              22
tenYearReturn               94
listedSh

In [ ]:
#mengisi NaN kolum general_info dengan string kosong
df['general_info'].fillna('', inplace=True)

In [ ]:
#string yang NaN ini diisi sama ''
df[['business_product','description', 'underwriter', 'shareholders']] = df[['business_product','description', 'underwriter', 'shareholders']].fillna('')

In [ ]:
count = df['_id'].nunique()
print(count)

883


In [ ]:
df.fillna(df.mean(), inplace=True) #mengisi dengan nilai rata-rata

<ipython-input-121-9598921fdd30>:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.fillna(df.mean(), inplace=True) #mengisi dengan nilai rata-rata


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 883 entries, 0 to 882
Data columns (total 54 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   _id                      883 non-null    object 
 1   sector                   883 non-null    object 
 2   idx                      883 non-null    object 
 3   thematic                 883 non-null    object 
 4   general_info             883 non-null    object 
 5   business_product         883 non-null    object 
 6   total_asset_growth       883 non-null    float64
 7   price_sales_ratio        883 non-null    float64
 8   sales_growth             883 non-null    float64
 9   quick_ratio              883 non-null    float64
 10  cost_of_goods_sold       883 non-null    float64
 11  cash_flow_per_share      883 non-null    float64
 12  net_income_growth        883 non-null    float64
 13  operating_profit         883 non-null    float64
 14  income_before_tax        8

In [ ]:
df.columns

Index(['_id', 'sector', 'idx', 'thematic', 'general_info', 'business_product',
       'total_asset_growth', 'price_sales_ratio', 'sales_growth',
       'quick_ratio', 'cost_of_goods_sold', 'cash_flow_per_share',
       'net_income_growth', 'operating_profit', 'income_before_tax',
       'total_assets', 'total_equity', 'total_liabilities',
       'debt_to_equity_ratio', 'cash_equivalent', 'total_sales',
       'operating_profit_margin', 'gross_profit_margin', 'net_profit_margin',
       'net_income', 'market_cap', 'dailyReturn', 'monthlyReturn',
       'annualReturn', 'threeYearReturn', 'fiveYearReturn', 'tenYearReturn',
       'listedShare', 'tradableVolume', 'offerVolume', 'tradableValue',
       'openPrice', 'closePrice', 'averagePrice', 'lastTradedPrice',
       'lowTradedPrice', 'highTradedPrice', 'bidPrice', 'offerPrice',
       'book_value_per_share', 'eps', 'pbv', 'roa', 'roe', 'per',
       'description', 'underwriter', 'shareholders', 'industry'],
      dtype='object')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 883 entries, 0 to 882
Data columns (total 54 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   _id                      883 non-null    object 
 1   sector                   883 non-null    object 
 2   idx                      883 non-null    object 
 3   thematic                 883 non-null    object 
 4   general_info             883 non-null    object 
 5   business_product         883 non-null    object 
 6   total_asset_growth       883 non-null    float64
 7   price_sales_ratio        883 non-null    float64
 8   sales_growth             883 non-null    float64
 9   quick_ratio              883 non-null    float64
 10  cost_of_goods_sold       883 non-null    float64
 11  cash_flow_per_share      883 non-null    float64
 12  net_income_growth        883 non-null    float64
 13  operating_profit         883 non-null    float64
 14  income_before_tax        8

In [ ]:
df.columns

Index(['_id', 'sector', 'idx', 'thematic', 'general_info', 'business_product',
       'total_asset_growth', 'price_sales_ratio', 'sales_growth',
       'quick_ratio', 'cost_of_goods_sold', 'cash_flow_per_share',
       'net_income_growth', 'operating_profit', 'income_before_tax',
       'total_assets', 'total_equity', 'total_liabilities',
       'debt_to_equity_ratio', 'cash_equivalent', 'total_sales',
       'operating_profit_margin', 'gross_profit_margin', 'net_profit_margin',
       'net_income', 'market_cap', 'dailyReturn', 'monthlyReturn',
       'annualReturn', 'threeYearReturn', 'fiveYearReturn', 'tenYearReturn',
       'listedShare', 'tradableVolume', 'offerVolume', 'tradableValue',
       'openPrice', 'closePrice', 'averagePrice', 'lastTradedPrice',
       'lowTradedPrice', 'highTradedPrice', 'bidPrice', 'offerPrice',
       'book_value_per_share', 'eps', 'pbv', 'roa', 'roe', 'per',
       'description', 'underwriter', 'shareholders', 'industry'],
      dtype='object')

# Menghapus kolum yang tidak digunakan (kali ini yaitu yang memiliki int)

In [ ]:
#aku akan menghapus semua yang float karena ini versi menggunakan bacotan okay
columns_to_drop = ['total_asset_growth', 'price_sales_ratio', 'sales_growth',
       'quick_ratio', 'cost_of_goods_sold', 'cash_flow_per_share',
       'net_income_growth', 'operating_profit', 'income_before_tax',
       'total_assets', 'total_equity', 'total_liabilities',
       'debt_to_equity_ratio', 'cash_equivalent', 'total_sales',
       'operating_profit_margin', 'gross_profit_margin', 'net_profit_margin',
       'net_income', 'market_cap', 'dailyReturn', 'monthlyReturn',
       'annualReturn', 'threeYearReturn', 'fiveYearReturn', 'tenYearReturn',
       'listedShare', 'tradableVolume', 'offerVolume', 'tradableValue',
       'openPrice', 'closePrice', 'averagePrice', 'lastTradedPrice',
       'lowTradedPrice', 'highTradedPrice', 'bidPrice', 'offerPrice',
       'book_value_per_share', 'eps', 'pbv', 'roa', 'roe', 'per']
df = df.drop(columns=columns_to_drop)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 883 entries, 0 to 882
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   _id               883 non-null    object
 1   sector            883 non-null    object
 2   idx               883 non-null    object
 3   thematic          883 non-null    object
 4   general_info      883 non-null    object
 5   business_product  883 non-null    object
 6   description       883 non-null    object
 7   underwriter       883 non-null    object
 8   shareholders      883 non-null    object
 9   industry          883 non-null    object
dtypes: object(10)
memory usage: 69.1+ KB


# Ayo rubah yang kode HTML

In [ ]:
#bacot ah HTML HTMLan, hapus, musnahkan
def html_to_string(html_code):
    soup = BeautifulSoup(html_code, 'html.parser')
    return soup.get_text()


In [ ]:
df['business_product'] = df['business_product'].apply(html_to_string)
df['description'] = df['description'].apply(html_to_string)
df['underwriter'] = df['underwriter'].apply(html_to_string)
df['shareholders'] = df['shareholders'].apply(html_to_string)

In [ ]:
df.head(2)

,_id,sector,idx,thematic,general_info,business_product,description,underwriter,shareholders,industry
0,AALI,"[""Non-Siklikal""]","[""IDXSMC-LIQ"" ""JII70"" ""ESGQKEHATI"" ""IDX80"" ""IS...",[],PT Astra Agro Lestari Tbk (IDX: AALI) adalah a...,Based on the Company's articles of association...,"Astra Agro Lestari, PT (the company) was estab...",PT. ABN AMRO Hoare Govett Indonesia,ShareholdersShares% PT. Astra International Tb...,Produk Makanan Pertanian
1,ABBA,"[""Siklikal""]",[],[],PT Mahaka Media Tbk (IDX: ABBA) merupakan peru...,,,,,Media


In [ ]:
print(df['underwriter'][0])
print(df['underwriter'][3])
print(df['underwriter'][100])
#underwiter menurut chatgpt adalah lembaga yang bertanggung jawab untuk memfasilitasi dan menyelenggarakan proses penawaran umum saham (IPO)
#atau penawaran sekuritas lainnya kepada publik
#jadi ini tidak perlu deh ya, hapus, musnahkan, hancurkan

 PT. ABN AMRO Hoare Govett Indonesia
PT Mandiri SekuritasPT Macquire Capital Securities Indonesia
 PT. Pacific CapitalPT. Panca Global SecuritiesPT. Universal Broker Indonesia


In [ ]:
print(df['shareholders'][0])
print(df['shareholders'][3])
print(df['shareholders'][100])
#Shareholder adalah pemilik saham dan memiliki kepemilikan atau bagian dari perusahaan yang dinyatakan dalam bentuk saham.
#terlihat seperti macam macam perusahaan pemilik saham perusahaan tersebut
#tidak ada hubungannya dengan apakah ada kaitan sama tujuan kita

ShareholdersShares% PT. Astra International Tbk153368244079.68 Public39100589320.32  As of Mar 31, 2021 
ShareholdersShares%Valle Verde Pte. Ltd. Singapura154055700055.55PT Tiara Marga Trakindo63636600023.07Momentum Fund SP.B28544730010.38Rachmat Mulyana Hamami61205000.55Achmad Ananda Djajanegara12835000.05Mivida Hamami1335000.01Public28325720010.41As of Mar 31, 2021
ShareholdersShares% PT Bumi Citra Investindo32728410022.89 Public : - PT Lodi Mitra Perkasa30979030021.66 - Domestic75463562552.77 - Foreign382055002.67  As of Jun 30, 2021 


In [ ]:
#hapus
columns_to_drop = ['underwriter','shareholders']
df = df.drop(columns=columns_to_drop)

In [ ]:
df.head()

,_id,sector,idx,thematic,general_info,business_product,description,industry
0,AALI,"[""Non-Siklikal""]","[""IDXSMC-LIQ"" ""JII70"" ""ESGQKEHATI"" ""IDX80"" ""IS...",[],PT Astra Agro Lestari Tbk (IDX: AALI) adalah a...,Based on the Company's articles of association...,"Astra Agro Lestari, PT (the company) was estab...",Produk Makanan Pertanian
1,ABBA,"[""Siklikal""]",[],[],PT Mahaka Media Tbk (IDX: ABBA) merupakan peru...,,,Media
2,ABDA,"[""Keuangan""]","[""IDXSMC-COM""]",[],PT Asuransi Bina Dana Arta Tbk (IDX: ABDA) mer...,The Company is engaged in general insurance bu...,PT Asuransi Bina Dana Arta Tbk was established...,Asuransi
3,ABMM,"[""Perindustrian""]","[""IDXSMC-COM"" ""KOMPAS100"" ""ISSI""]","[""Value Investing"" ""Komoditas Supercycle""]",PT ABM Investama Tbk (IDX: ABMM) merupakan emi...,The Company's scope of activities are conducti...,"ABM Investama, PT (the company) was establishe...",Perusahaan Holding Multi-sektor
4,ACES,"[""Siklikal""]","[""IDXSMC-LIQ"" ""JII"" ""IDX80"" ""IDXQ30"" ""ISSI"" ""I...","[""Retailer Indonesia"" ""ESG Leaders""]",PT Ace Hardware Indonesia Tbk (IDX: ACES) adal...,The Company's scope of activities consist of g...,PT Ace Hardware Indonesia (the Company) was es...,Ritel Khusus


# Pengolahan Datafreme Lebih lanjut saudara saudara, mencari key words

In [ ]:
df.set_index('_id', inplace = True)
df.head()

KeyError: ignored

In [ ]:
df['general_info'][10]

'Polychem Indonesia Tbk (IDX: ADMG) adalah perusahaan yang memiliki bidang usaha utama di industri Kimia dan Tekstil, dengan memproduksi dan mengekspor barang Kimia Dasar. Pendapatan utama perusahaan berasal dari penjualan polyester, juga berasal dari Ethylene Glycol dan petrokimia. Adapun PT Gajah Tunggal Tbk (IDX: GJTL) tercatat memiliki porsi kepemilikan saham sebesar 26 persen di perusahaan yang beralamat kantor di Gedung Wisma 46 Kota BNI LT. 20 Jl Jend. Sudirman Kav 1 RT. 010 RW. 009 Karet Tengsin Tanah Abang Kota Adm. Jakarta Pusat DKI Jakarta 10220. Perusahaan resmi listing di BEI pada tanggal 20 Oktober 1993.'

In [ ]:
df['business_product'][19]

'The current business activities of the Company and its subsidiaries include business and trade industry noodle, especially noodles dried, instant noodles, and rice noodles, snack, biscuit industry, sweets, oil palm plantations, power generation, processing and distribution of rice.'

In [ ]:
df['description'][19]

'PT Tiga Pilar Sejahtera Tbk formerly PT Asia Intiselera Tbk was established on January 26, 1990. PT. Asia Intiselera is a producer of Instant Noodles, this company was founded by Kang family under the name Pabrik Mie Asia which located in Jakarta. the company started its operations since 1953, and in 1974 the factory moved to Bogor. The articles of association have been amended several times, most recently on January 25, 1997, concerning the change of status and increase in authorized capital.'

In [ ]:
df.head(1)

,sector,idx,thematic,general_info,business_product,description,industry
_id,,,,,,,
AALI,"[""Non-Siklikal""]","[""IDXSMC-LIQ"" ""JII70"" ""ESGQKEHATI"" ""IDX80"" ""IS...",[],PT Astra Agro Lestari Tbk (IDX: AALI) adalah a...,Based on the Company's articles of association...,"Astra Agro Lestari, PT (the company) was estab...",Produk Makanan Pertanian


In [ ]:
df['key_words_business_product'] = ''  # Menetapkan nilai awal untuk kolom 'key_words'

for index, row in df.iterrows():
    plot = row['business_product']
    r = Rake()
    r.extract_keywords_from_text(plot)
    key_words_dict_scores = r.get_word_degrees()
    df.at[index, 'key_words_business_product'] = list(key_words_dict_scores.keys())



In [ ]:
df['key_words_general_info'] = ''  # Menetapkan nilai awal untuk kolom 'key_words'

for index, row in df.iterrows():
    plot = row['general_info']
    r = Rake(language='indonesian')
    r.extract_keywords_from_text(plot)
    key_words_dict_scores = r.get_word_degrees()
    df.at[index, 'key_words_general_info'] = list(key_words_dict_scores.keys())


In [ ]:
df['key_words_description'] = ''  # Menetapkan nilai awal untuk kolom 'key_words'

for index, row in df.iterrows():
    plot = row['description']
    r = Rake(language='indonesian')
    r.extract_keywords_from_text(plot)
    key_words_dict_scores = r.get_word_degrees()
    df.at[index, 'key_words_description'] = list(key_words_dict_scores.keys())

In [ ]:
columns_to_drop = ['general_info','business_product','description']
df = df.drop(columns=columns_to_drop)

In [ ]:
df.head()

,sector,idx,thematic,industry,key_words_business_product,key_words_general_info,key_words_description
_id,,,,,,,
AALI,"[""Non-Siklikal""]","[""IDXSMC-LIQ"" ""JII70"" ""ESGQKEHATI"" ""IDX80"" ""IS...",[],Produk Makanan Pertanian,"[based, company, articles, association, scope,...","[pt, astra, agro, lestari, tbk, idx, aali, ana...","[astra, agro, lestari, pt, the, company, was, ..."
ABBA,"[""Siklikal""]",[],[],Media,[],"[pt, mahaka, media, tbk, idx, abba, perusahaan...",[]
ABDA,"[""Keuangan""]","[""IDXSMC-COM""]",[],Asuransi,"[company, engaged, general, insurance, busines...","[pt, asuransi, bina, dana, arta, tbk, idx, abd...","[pt, asuransi, bina, dana, arta, tbk, was, est..."
ABMM,"[""Perindustrian""]","[""IDXSMC-COM"" ""KOMPAS100"" ""ISSI""]","[""Value Investing"" ""Komoditas Supercycle""]",Perusahaan Holding Multi-sektor,"[company, scope, activities, conducting, busin...","[pt, abm, investama, tbk, idx, abmm, emiten, m...","[abm, investama, pt, the, company, was, establ..."
ACES,"[""Siklikal""]","[""IDXSMC-LIQ"" ""JII"" ""IDX80"" ""IDXQ30"" ""ISSI"" ""I...","[""Retailer Indonesia"" ""ESG Leaders""]",Ritel Khusus,"[company, scope, activities, consist, general,...","[pt, ace, hardware, indonesia, tbk, idx, aces,...","[pt, ace, hardware, indonesia, the, company, w..."


In [ ]:
#idx dihapus saja karena itu terkait saham itu di list dimana saja di bursa, tidak ada kalitannya sama rekomendation system
columns_to_drop = ['idx']
df = df.drop(columns=columns_to_drop)

In [ ]:
df['thematic'] = df['thematic'].str.replace('[', '').str.replace('"', '').str.replace(']', '')

<ipython-input-146-8b033f66425a>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['thematic'] = df['thematic'].str.replace('[', '').str.replace('"', '').str.replace(']', '')


In [ ]:
df['industry'] = df['industry'].map(lambda x:x.split(' '))

In [ ]:
for index, row in df.iterrows():
  row['industry'] = ''.join(row['industry']).lower()

In [ ]:
df.head(1)

,sector,thematic,industry,key_words_business_product,key_words_general_info,key_words_description,bag_of_words
_id,,,,,,,
AALI,non-siklikal,,[produkmakananpertanian],"[based, company, articles, association, scope,...","[pt, astra, agro, lestari, tbk, idx, aali, ana...","[astra, agro, lestari, pt, the, company, was, ...",n o n - s i k l i k a l produkmakananpertania...


In [ ]:
df['industry'] = df['industry'].map(lambda x : x.lower().split(' '))

In [ ]:
type(df['sector'])

pandas.core.series.Series

In [ ]:
df['sector'] = df['sector'].str.replace('[', '').str.replace('"', '').str.replace(']', '')

<ipython-input-151-b0b9ecc882b9>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['sector'] = df['sector'].str.replace('[', '').str.replace('"', '').str.replace(']', '')


In [ ]:
df['sector'] = df['sector'].str.lower()

In [ ]:
df['bag_of_words'] = ''
columns = df.columns
for index, row in df.iterrows():
  words= ''
  for col in columns:
      words = words + ' '.join(row[col]) + ' '

  row['bag_of_words'] = words


In [ ]:
#mengecek ada berapa baris yang mengandung double space
df['bag_of_words'].str.contains(r'\s\s').sum()

0

In [ ]:
df['bag_of_words'] = df['bag_of_words'].str.replace(r'\s\s', ' ')

<ipython-input-185-df9588f9567a>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['bag_of_words'] = df['bag_of_words'].str.replace(r'\s\s', ' ')


pandas.core.series.Series

In [ ]:
df.head()

,bag_of_words
_id,
AALI,non-siklikal idxsmc-liq jii70 esgqkehati idx80...
ABBA,siklikal media pt mahaka media tbk idx abba pe...
ABDA,keuangan idxsmc-com asuransi company engaged g...
ABMM,perindustrian idxsmc-com kompas100 issi V a l ...
ACES,siklikal idxsmc-liq jii idx80 idxq30 issi idxe...


In [ ]:
count = CountVectorizer()
count_matrix = count.fit_transform(df['bag_of_words'])

In [ ]:
count_matrix

<883x9426 sparse matrix of type '<class 'numpy.int64'>'
	with 60989 stored elements in Compressed Sparse Row format>

In [ ]:
c = count_matrix.todense()
c

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 1, 1, 1]])

In [ ]:
print(count_matrix[0,:])

  (0, 5979)	1
  (0, 7975)	1
  (0, 3906)	2
  (0, 5044)	1
  (0, 4326)	1
  (0, 3093)	1
  (0, 3899)	1
  (0, 4207)	1
  (0, 4161)	1
  (0, 4724)	1
  (0, 8209)	1
  (0, 3907)	1
  (0, 3094)	1
  (0, 2256)	1
  (0, 7104)	1
  (0, 1333)	1
  (0, 2279)	1
  (0, 1117)	1
  (0, 1176)	1
  (0, 7746)	1
  (0, 716)	1
  (0, 3037)	1
  (0, 6866)	1
  (0, 6120)	1
  (0, 3505)	1
  :	:
  (0, 5690)	1
  (0, 7722)	1
  (0, 6241)	1
  (0, 6065)	1
  (0, 6492)	1
  (0, 4541)	1
  (0, 5473)	1
  (0, 8513)	1
  (0, 7107)	1
  (0, 8593)	1
  (0, 4391)	1
  (0, 5451)	1
  (0, 8360)	1
  (0, 4406)	1
  (0, 8350)	1
  (0, 1309)	1
  (0, 7493)	1
  (0, 1326)	1
  (0, 7462)	1
  (0, 5054)	1
  (0, 1411)	1
  (0, 8521)	1
  (0, 32)	1
  (0, 2611)	1
  (0, 320)	1


In [ ]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[1.        , 0.24802326, 0.26595839, ..., 0.23865177, 0.23788041,
        0.28248299],
       [0.24802326, 1.        , 0.3207008 , ..., 0.21637116, 0.15097027,
        0.20758412],
       [0.26595839, 0.3207008 , 1.        , ..., 0.24980546, 0.20640627,
        0.24080732],
       ...,
       [0.23865177, 0.21637116, 0.24980546, ..., 1.        , 0.23519241,
        0.29399052],
       [0.23788041, 0.15097027, 0.20640627, ..., 0.23519241, 1.        ,
        0.21794872],
       [0.28248299, 0.20758412, 0.24080732, ..., 0.29399052, 0.21794872,
        1.        ]])

In [ ]:
indices = pd.Series(df.index)
indices

0      AALI
1      ABBA
2      ABDA
3      ABMM
4      ACES
       ... 
878    ZATA
879    ZBRA
880    ZINC
881    ZONE
882    ZYRX
Name: _id, Length: 883, dtype: object

In [ ]:
def recomendations(id, cosine_sim = cosine_sim):
  recomended_stock= []
  idx = indices[indices == id].index[0]

  score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

  top_10_indexes = list(score_series.iloc[1:11].index)

  print(top_10_indexes)

  for i in top_10_indexes:
    recomended_stock.append(list(df.index)[i])

  return recomended_stock

In [ ]:
recomendations('TAPG')

[802, 454, 0, 656, 177, 490, 765, 256, 495, 798]


['TLDN',
 'KMTR',
 'AALI',
 'PSGO',
 'BWPT',
 'LSIP',
 'SSMS',
 'DSNG',
 'MAGP',
 'TINS']